In [29]:
!pip install --user scikit-learn==0.22.2

  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22
    Uninstalling scikit-learn-0.22:
      Successfully uninstalled scikit-learn-0.22


In [1]:
# !pip install tensorflow==1.1.0

# !python -m spacy download de
import nltk
# nltk.download('punkt')
import tensorflow as tf
import pickle
import os
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline 
import string
import spacy
from spacy.lang.de.stop_words import STOP_WORDS
from spacy.lang.de import German
import re
import numpy as np

print('TensorFlow Version: {}'.format(tf.__version__))

C:\Users\DEEPANSHU SONPAROTE\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\DEEPANSHU SONPAROTE\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\DEEPANSHU SONPAROTE\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\D

TensorFlow Version: 1.14.0


In [2]:
# TEXT SUMMARIZATION FUNCTION
def summary_func():
    input_sentence = textName1.get()
    batch_size = 64
    with open('vocabdict','rb') as f:
        vocab_to_int = pickle.load(f)

    with open('intdict','rb') as f:
        int_to_vocab = pickle.load(f)

    def clean_text(text):
      '''Remove unwanted characters, stopwords, and format the text to create fewer nulls word embeddings'''

      # Convert words to lower case
      text = text.lower()

      # Format words and remove unwanted characters
      text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
      text = re.sub(r'\<a href', ' ', text)
      text = re.sub(r'&amp;', '', text) 
      text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
      text = re.sub(r'<br />', ' ', text)
      text = re.sub(r'\'', ' ', text)
      return text

    def text_to_seq(text):
      '''Prepare the text for the model'''
      with open('vocabdict','rb') as f:
        vocab_to_int = pickle.load(f)

      text = clean_text(text)
      return [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in text.split()]

    text = text_to_seq(input_sentence)
    checkpoint = "./best_model.ckpt"

    loaded_graph = tf.Graph()
    with tf.Session(graph=loaded_graph) as sess:
      # Load saved model
      loader = tf.train.import_meta_graph(checkpoint + '.meta')
      loader.restore(sess, checkpoint)

      input_data = loaded_graph.get_tensor_by_name('input:0')
      logits = loaded_graph.get_tensor_by_name('predictions:0')
      text_length = loaded_graph.get_tensor_by_name('text_length:0')
      summary_length = loaded_graph.get_tensor_by_name('summary_length:0')
      keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

      #Multiply by batch_size to match the model's input parameters
      answer_logits = sess.run(logits, {input_data: [text]*batch_size, 
                                        summary_length: [np.random.randint(5,8)], 
                                        text_length: [len(text)]*batch_size,
                                        keep_prob: 1.0})[0] 

    # Remove the padding from the tweet
    pad = vocab_to_int["<PAD>"] 
    textName2.insert(0," ".join([int_to_vocab[i] for i in answer_logits if i != pad]))


In [3]:

from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

punctuations = string.punctuation
nlp = spacy.load('de_core_news_sm')
stop_words = spacy.lang.de.stop_words.STOP_WORDS
parser = German()

def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]
    return mytokens

class predictors(TransformerMixin):
    def transform(self, X, **transform_params): 
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

def clean_text(text):
    return text.strip().lower()

with open('logistic_sentiment','rb') as f:
    pipe_log = pickle.load(f)

C:\Users\DEEPANSHU SONPAROTE\AppData\Roaming\Python\Python36\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.2.post1 when using version 0.22.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\DEEPANSHU SONPAROTE\AppData\Roaming\Python\Python36\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.22.2.post1 when using version 0.22.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\DEEPANSHU SONPAROTE\AppData\Roaming\Python\Python36\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator Pipeline from version 0.22.2.post1 when using version 0.22.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [7]:
def sentiment_func():
    input_sentence = textName3.get()

    k = pipe_log.predict([input_sentence])
    textName4.delete(0,END) 
    textName4.insert(0,str(k[0]))


In [8]:
'''
GUI -> 2Sections:
1) Big Text to Summarized Text Converter
2) Text Sentiment Analyser
3)

'''

from ttkthemes import themed_tk as tk
from tkinter import *
from tkinter import filedialog, ttk
from PIL import ImageTk,Image
from tkinter import messagebox

root = tk.ThemedTk()
root.get_themes()
root.set_theme('breeze')

windowWidth = 600
windowHeight = 300
positionRight = int(root.winfo_screenwidth() / 2 - windowWidth / 2)
positionDown = int(root.winfo_screenheight() / 2 - windowHeight / 2)

inputVar = StringVar()
inputVar.set('Loremaaaaadvscvascikchasnoilasconsoiclnsilcnsaocnlunsocilasnclijscionscousjncasicadskubciuascahoisabnuioch')

class Page(ttk.Frame):
    def __init__(self, *args, **kwargs):
        ttk.Frame.__init__(self, *args, **kwargs)
    def show(self):
        self.lift()

class Page1(Page):
    
    def __init__(self, *args, **kwargs):
        Page.__init__(self, *args, **kwargs)
        try:
            im = Image.open("media/amz.png")
            # im = im.convert("RGBA")
            # im = im.resize((600, 100))
            photo = ImageTk.PhotoImage(im)

            logo = Label(self, image=photo)
            logo.image = photo  # keep a reference!
            logo.pack()
        except:
            messagebox.showerror("Error", 'Resource Missing.\nThe software will quit once you click the button')
            global root
            root.destroy()
            return

        X = 40
        Y = 60
        global textName1, textName2
        MainLabel = ttk.Label(self, text = 'Summarize Text', width = 170, font = "Helvetica 24 bold", anchor="center").pack(side=TOP)

        Y = Y + 40

        labelName1 = ttk.Label(self, text = 'Enter Paragraph', width = 15, justify = RIGHT, anchor = 'e')
        labelName1.place(x = X, y = Y)

        textName1 = ttk.Entry(self, width=40)
        textName1.place(x = X + 120, y = Y)

        Y = Y + 40
        buttonBrowse1 = ttk.Button(self, text = "Summarize", command =summary_func)
        buttonBrowse1.place(x = X + 200, y = Y + 2)

        Y = Y + 60

        labelName1 = ttk.Label(self, text = 'Summary', width = 15, justify = RIGHT, anchor = 'e')
        labelName1.place(x = X, y = Y)

        textName2 = ttk.Entry(self,  width=40)
        textName2.place(x = X + 120, y = Y)
        



class Page2(Page):
    def __init__(self, *args, **kwargs):
        Page.__init__(self, *args, **kwargs)

        try:
            im = Image.open("media/amz.png")
            # im = im.convert("RGBA")
            # im = im.resize((600, 100))
            photo = ImageTk.PhotoImage(im)

            logo = Label(self, image=photo)
            logo.image = photo  # keep a reference!
            logo.pack()
        except:
            messagebox.showerror("Error", 'Resource Missing.\nThe software will quit once you click the button')
            global root
            root.destroy()
            return

        X = 40
        Y = 60
        global textName3, textName4
        MainLabel = ttk.Label(self, text = 'Sentiment Analysis', width = 200, font = "Helvetica 24 bold", anchor="center").pack()

        Y = Y + 40

        labelName1 = ttk.Label(self, text = 'Enter Sentence', width = 15, justify = LEFT, anchor = 'e')
        labelName1.place(x = X, y = Y)
        textName3 = ttk.Entry(self, width=40)
        textName3.place(x = X + 120, y = Y)

        Y = Y + 40
        buttonBrowse1 = ttk.Button(self, text = "Analyse", command =sentiment_func)
        buttonBrowse1.place(x = X + 200, y = Y + 2)

        Y = Y + 60

        labelName1 = ttk.Label(self, text = 'Sentiment', width = 15, justify = LEFT, anchor = 'e')
        labelName1.place(x = X, y = Y)

        textName4 = ttk.Entry(self, width=40)
        textName4.place(x = X + 120, y = Y)

# class Page3(Page):
#     def __init__(self, *args, **kwargs):
#         Page.__init__(self, *args, **kwargs)


class MainView(Frame):
    def __init__(self, *args, **kwargs):
        Frame.__init__(self, *args, **kwargs)
        p1 = Page1(self)
        p2 = Page2(self)
        # p3 = Page3(self)

        buttonframe = ttk.Frame(self)
        container = ttk.Frame(self)
        buttonframe.pack(side= LEFT, fill="both", expand=False)
        container.pack(side=LEFT, fill="both", expand=True)

        p1.place(in_=container, x=0, y=0, relwidth=1, relheight=1)
        p2.place(in_=container, x=0, y=0, relwidth=1, relheight=1)
        # p3.place(in_=container, x=0, y=0, relwidth=1, relheight=1)

        b1 = ttk.Button(buttonframe, text="Summarize", command=p1.lift)
        # b1.config(bd=5, relief=RAISED)
        b2 = ttk.Button(buttonframe, text="Analyse", command=p2.lift)
        # b2.config(bd=5, relief=RAISED)
        # b3 = Button(buttonframe, text="Page 3", command=p3.lift)
        # b3.config(bd=5, relief=RAISED)

        b1.pack(side = TOP, expand = True, fill = BOTH)
        b2.pack(side = TOP, expand = True, fill = BOTH)
        # b3.pack(side = TOP, expand = True, fill = BOTH)

        p1.show()

def main_function():
    main = MainView(root)

    # Gets the requested values of the height and widht.

    main.pack(side="top", fill="both", expand=True)

    root.title('Sentiment Analyis on Abstractive Summarization')
    root.geometry("{}x{}+{}+{}".format(windowWidth, windowHeight, positionRight, positionDown))


    root.mainloop()

if __name__ == "__main__":

    main = MainView(root)

    # Gets the requested values of the height and widht.

    main.pack(side="top", fill="both", expand=True)

    root.title('Sentiment Analyis on Abstractive Summarization')
    root.geometry("{}x{}+{}+{}".format(windowWidth, windowHeight, positionRight, positionDown))
    root.resizable(False, False)

    root.mainloop()

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-tk is already the newest version (3.6.9-1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


In [3]:
!pip install ttkthemes

  Created wheel for ttkthemes: filename=ttkthemes-3.0.0-cp36-cp36m-win_amd64.whl size=1784119 sha256=238620f246733a5579d6987e7cf7618d8965defd2cf9b47956195d0640115b3d
  Stored in directory: c:\users\deepanshu sonparote\appdata\local\pip\cache\wheels\2b\c1\83\25b49b153e06dd8d1516acc738adb4ecd078018cb20996f181
Successfully built ttkthemes
